In [6]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
import math

In [7]:
TRANSFORMED_DATA_PATH = "results/trajectories/transformed_coords.csv"

In [8]:
df = pd.read_csv(TRANSFORMED_DATA_PATH)
df.head()

,Unnamed: 0.1,Unnamed: 0,video_name,frame_id,xmin,ymin,xmax,ymax,velocity,customer_id,x_floorplan,y_floorplan,camera,x_floorplan_new,y_floorplan_new,time,customer_id_global
0,0,0,20220314_1.03_10_0_blurred,258,653.596516,233.180269,732.845525,330.121294,0.000,2383,-359.594824,-48.170086,1.03,1226.894568,1430.999265,14:47:33,2383.0
1,1,1,20220314_1.03_10_0_blurred,260,653.706260,233.044283,732.741128,329.715728,0.105,2383,-359.589552,-48.161038,1.03,1227.066483,1431.227170,14:47:33,2383.0
2,2,2,20220314_1.03_10_0_blurred,261,653.599622,233.117391,731.924925,328.843053,0.185,2383,-359.588172,-48.152324,1.03,1227.111485,1431.446655,14:47:33,2383.0
3,3,3,20220314_1.03_10_0_blurred,264,653.864126,233.172724,732.331825,329.024946,0.128,2383,-359.587332,-48.155114,1.03,1227.138867,1431.376397,14:47:34,2383.0
4,4,4,20220314_1.03_10_0_blurred,285,653.823460,232.950344,732.490189,329.088096,0.119,2383,-359.586592,-48.153137,1.03,1227.163013,1431.426174,14:47:37,2383.0


In [9]:
def re_id_customers(df):
    for customer in df.customer_id_global.unique():
        if customer == 1191:
            last_customer_row = df.loc[df["customer_id_global"] == customer].sort_values(by='time').iloc[-1]
            print(last_customer_row)
            
            df_similar_time_customers = get_similar_time_customers(df, last_customer_row["time"], 5, last_customer_row["camera"])
            if df_similar_time_customers.empty:
                print("EMPTY 1")
                continue

            #print(df_similar_time_customers)

            similar_position_customers = get_close_customers(df_similar_time_customers, last_customer_row["x_floorplan_new"], last_customer_row["y_floorplan_new"], 100)
            if similar_position_customers.empty:
                print("EMPTY 2")
                continue

            similar_position_customers.head()
            #df.loc[similar_position_customers, "customer_reid"] = last_customer_row["customer_id_global"]
            #similar_position_customers["customer_reid"] = last_customer_row["customer_id_global"]
            #df.head()
            break


def get_similar_time_customers(df, time, interval, camera):
    df_other_cameras = df.loc[df["camera"] != camera]
    print(df_other_cameras)
    begin_time = str((datetime.strptime(time,"%H:%M:%S") - timedelta(seconds=interval)).time())
    print(begin_time)
    end_time = str((datetime.strptime(time,"%H:%M:%S") + timedelta(seconds=interval)).time())
    print(end_time)
    df_time_interval = get_data_by_time(df_other_cameras, begin_time, end_time)
    return df_time_interval


def get_close_customers(df, xcoord, ycoord, max_dist):
    df_close_customers = df[df.apply(lambda row: math.dist((xcoord, ycoord), (row["x_floorplan_new"], row["y_floorplan_new"])), axis=1) < max_dist]          
    return df_close_customers

    
def get_data_by_time(df, begin_time, end_time):
    return df.loc[(df["time"] > begin_time) & (df["time"] < end_time)]


In [10]:
re_id_customers(df)

Unnamed: 0.1                               175350
Unnamed: 0                                     47
video_name            20220314_1.03_23_13_blurred
frame_id                                      152
xmin                                  1333.193071
ymin                                    61.404172
xmax                                   1398.15771
ymax                                    239.74064
velocity                                      0.0
customer_id                                  1191
x_floorplan                             -354.2079
y_floorplan                             -51.98757
camera                                       1.03
x_floorplan_new                       1402.566385
y_floorplan_new                       1334.843599
time                                     22:56:19
customer_id_global                         1191.0
Name: 175350, dtype: object
        Unnamed: 0.1  Unnamed: 0                 video_name  frame_id  \
227940        227940           0  20220314_1.04_1